In [1]:
import gensim, logging

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
gmodel = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

2017-11-12 16:06:33,788 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin
2017-11-12 16:08:10,091 : INFO : loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin


In [4]:
gmodel['cat']

array([ 0.0123291 ,  0.20410156, -0.28515625,  0.21679688,  0.11816406,
        0.08300781,  0.04980469, -0.00952148,  0.22070312, -0.12597656,
        0.08056641, -0.5859375 , -0.00445557, -0.296875  , -0.01312256,
       -0.08349609,  0.05053711,  0.15136719, -0.44921875, -0.0135498 ,
        0.21484375, -0.14746094,  0.22460938, -0.125     , -0.09716797,
        0.24902344, -0.2890625 ,  0.36523438,  0.41210938, -0.0859375 ,
       -0.07861328, -0.19726562, -0.09082031, -0.14160156, -0.10253906,
        0.13085938, -0.00346375,  0.07226562,  0.04418945,  0.34570312,
        0.07470703, -0.11230469,  0.06738281,  0.11230469,  0.01977539,
       -0.12353516,  0.20996094, -0.07226562, -0.02783203,  0.05541992,
       -0.33398438,  0.08544922,  0.34375   ,  0.13964844,  0.04931641,
       -0.13476562,  0.16308594, -0.37304688,  0.39648438,  0.10693359,
        0.22167969,  0.21289062, -0.08984375,  0.20703125,  0.08935547,
       -0.08251953,  0.05957031,  0.10205078, -0.19238281, -0.09

In [5]:
gmodel['dog']

array([  5.12695312e-02,  -2.23388672e-02,  -1.72851562e-01,
         1.61132812e-01,  -8.44726562e-02,   5.73730469e-02,
         5.85937500e-02,  -8.25195312e-02,  -1.53808594e-02,
        -6.34765625e-02,   1.79687500e-01,  -4.23828125e-01,
        -2.25830078e-02,  -1.66015625e-01,  -2.51464844e-02,
         1.07421875e-01,  -1.99218750e-01,   1.59179688e-01,
        -1.87500000e-01,  -1.20117188e-01,   1.55273438e-01,
        -9.91210938e-02,   1.42578125e-01,  -1.64062500e-01,
        -8.93554688e-02,   2.00195312e-01,  -1.49414062e-01,
         3.20312500e-01,   3.28125000e-01,   2.44140625e-02,
        -9.71679688e-02,  -8.20312500e-02,  -3.63769531e-02,
        -8.59375000e-02,  -9.86328125e-02,   7.78198242e-03,
        -1.34277344e-02,   5.27343750e-02,   1.48437500e-01,
         3.33984375e-01,   1.66015625e-02,  -2.12890625e-01,
        -1.50756836e-02,   5.24902344e-02,  -1.07421875e-01,
        -8.88671875e-02,   2.49023438e-01,  -7.03125000e-02,
        -1.59912109e-02,

In [6]:
gmodel['spatula']

array([-0.19140625, -0.04296875,  0.27539062,  0.00488281, -0.3203125 ,
        0.08203125,  0.05566406, -0.03613281, -0.31445312,  0.10693359,
       -0.359375  ,  0.29882812,  0.02331543,  0.05517578, -0.140625  ,
        0.1953125 , -0.23632812, -0.22167969, -0.06542969, -0.3359375 ,
        0.25195312, -0.09326172,  0.54296875,  0.11328125, -0.28710938,
       -0.12011719, -0.11181641,  0.20996094, -0.33203125,  0.30273438,
       -0.3359375 , -0.12255859,  0.12890625, -0.28515625, -0.04223633,
        0.25585938,  0.3203125 ,  0.07177734,  0.19042969, -0.01379395,
        0.16992188, -0.22460938,  0.5078125 ,  0.08398438, -0.07519531,
       -0.06396484,  0.05371094,  0.34570312,  0.46289062, -0.16699219,
       -0.30664062,  0.15234375, -0.09765625, -0.26171875, -0.14160156,
        0.2265625 ,  0.49609375, -0.10791016, -0.08447266,  0.234375  ,
        0.04931641, -0.07128906,  0.05273438, -0.11914062,  0.09814453,
        0.11181641, -0.13574219, -0.46875   ,  0.26171875,  0.12

In [7]:
gmodel.similarity('cat', 'dog')

0.76094570897822089

In [8]:
gmodel.similarity('cat', 'spatula')

0.12412612600429632

In [9]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [10]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) # strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remove apostrophes
    sent = re.sub(r'\W', ' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated spaces
    sent = sent.strip()
    return sent.split()

In [11]:
# unsupervised training data
import re
import os
unsup_sentences = []

# source: http://ai.stanford.edu/~amaas/data/sentiment/, data from IMDB
for dirname in ["train/pos", "train/neg", "train/unsup", "test/pos", "test/neg"]:
    for fname in sorted(os.listdir("aclImdb/" + dirname)):
        if fname[-4:] == '.txt':
            with open("aclImdb/" + dirname + "/" + fname, encoding='UTF-8') as f:
                sent = f.read()
                words = extract_words(sent)
                unsup_sentences.append(TaggedDocument(words, [dirname + "/" + fname]))

# source: http://www.cs.cornell.edu/people/pabo/movie-review-data/
for dirname in ["review_polarity/txt_sentoken/pos", "review_polarity/txt_sentoken/neg"]:
    for fname in sorted(os.listdir(dirname)):
        if fname[-4:] == '.txt':
            with open(dirname + "/" + fname, encoding='UTF-8') as f:
                for i, sent in enumerate(f):
                    words = extract_words(sent)
                    unsup_sentences.append(TaggedDocument(words, ["%s/%s-%d" % (dirname, fname, i)]))
                
# source: https://nlp.stanford.edu/sentiment/, data from Rotten Tomatoes
with open("stanfordSentimentTreebank/original_rt_snippets.txt", encoding='UTF-8') as f:
    for i, line in enumerate(f):
        words = extract_words(sent)
        unsup_sentences.append(TaggedDocument(words, ["rt-%d" % i]))

In [12]:
len(unsup_sentences)

175325

In [13]:
unsup_sentences[0:10]

[TaggedDocument(words=['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'hig', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', 'the', 'scramble', 'to', 'survive', 'financially', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', 'i', 'immediately', 'recalled', 'at', 'high', 'a', 'classic', 'line', 'inspector', 'here', 'to', 'sack', 'one', 'of', 'your', 'teachers', 'student', 'welcome', 

In [14]:
import random
class PermuteSentences(object):
    def __init__(self, sents):
        self.sents = sents
        
    def __iter__(self):
        shuffled = list(self.sents)
        random.shuffle(shuffled)
        for sent in shuffled:
            yield sent

In [15]:
permuter = PermuteSentences(unsup_sentences) 
model = Doc2Vec(permuter, dm=0, hs=1, size=50)

2017-11-12 16:09:07,700 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-11-12 16:09:07,703 : INFO : collecting all words and their counts
2017-11-12 16:09:08,097 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-11-12 16:09:08,837 : INFO : PROGRESS: at example #10000, processed 1389399 words (1883036/s), 44788 word types, 10000 tags
2017-11-12 16:09:09,546 : INFO : PROGRESS: at example #20000, processed 2818609 words (2024079/s), 61270 word types, 20000 tags
2017-11-12 16:09:10,234 : INFO : PROGRESS: at example #30000, processed 4225040 words (2051374/s), 72774 word types, 30000 tags
2017-11-12 16:09:10,921 : INFO : PROGRESS: at example #40000, processed 5632885 words (2057947/s), 82225 word types, 40000 tags
2017-11-12 16:09:11,602 : INFO : PROGRESS: at example #50000, processed 7040997 words (2074544/s), 90454 word types, 50000 tags
2017-11-12 16:09:12,284 : INFO : PROGRESS: at example #60000, processed 847450

2017-11-12 16:10:19,029 : INFO : PROGRESS: at 15.44% examples, 308818 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:10:20,079 : INFO : PROGRESS: at 15.79% examples, 308916 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:10:21,116 : INFO : PROGRESS: at 16.13% examples, 309076 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:10:22,128 : INFO : PROGRESS: at 16.46% examples, 309106 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:10:23,180 : INFO : PROGRESS: at 16.81% examples, 309171 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:10:24,186 : INFO : PROGRESS: at 17.14% examples, 309106 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:10:25,200 : INFO : PROGRESS: at 17.48% examples, 309241 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:10:26,214 : INFO : PROGRESS: at 17.82% examples, 309239 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:10:27,226 : INFO : PROGRESS: at 18.15% examples, 309143 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:10:28,253 : INFO : PROGRESS: at 18.50% examples, 309226 wor

2017-11-12 16:11:40,406 : INFO : PROGRESS: at 42.67% examples, 311884 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:11:41,410 : INFO : PROGRESS: at 43.00% examples, 311833 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:11:42,455 : INFO : PROGRESS: at 43.34% examples, 311747 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:11:43,464 : INFO : PROGRESS: at 43.68% examples, 311687 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:11:44,478 : INFO : PROGRESS: at 44.00% examples, 311598 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:11:45,491 : INFO : PROGRESS: at 44.33% examples, 311584 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:11:46,496 : INFO : PROGRESS: at 44.65% examples, 311533 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:11:47,516 : INFO : PROGRESS: at 44.98% examples, 311501 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:11:48,516 : INFO : PROGRESS: at 45.31% examples, 311455 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:11:49,524 : INFO : PROGRESS: at 45.64% examples, 311393 wor

2017-11-12 16:13:01,723 : INFO : PROGRESS: at 69.20% examples, 309686 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:13:02,745 : INFO : PROGRESS: at 69.53% examples, 309709 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:13:03,763 : INFO : PROGRESS: at 69.88% examples, 309708 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:13:04,775 : INFO : PROGRESS: at 70.20% examples, 309705 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:13:05,776 : INFO : PROGRESS: at 70.54% examples, 309719 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:13:06,791 : INFO : PROGRESS: at 70.88% examples, 309711 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:13:07,803 : INFO : PROGRESS: at 71.21% examples, 309678 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:13:08,821 : INFO : PROGRESS: at 71.54% examples, 309701 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:13:09,829 : INFO : PROGRESS: at 71.87% examples, 309670 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:13:10,830 : INFO : PROGRESS: at 72.20% examples, 309652 wor

2017-11-12 16:14:23,596 : INFO : PROGRESS: at 95.97% examples, 308704 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:14:24,604 : INFO : PROGRESS: at 96.31% examples, 308716 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:14:25,605 : INFO : PROGRESS: at 96.63% examples, 308702 words/s, in_qsize 6, out_qsize 0
2017-11-12 16:14:26,610 : INFO : PROGRESS: at 96.97% examples, 308714 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:14:27,629 : INFO : PROGRESS: at 97.30% examples, 308709 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:14:28,643 : INFO : PROGRESS: at 97.64% examples, 308710 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:14:29,644 : INFO : PROGRESS: at 97.96% examples, 308699 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:14:30,650 : INFO : PROGRESS: at 98.29% examples, 308711 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:14:31,660 : INFO : PROGRESS: at 98.62% examples, 308744 words/s, in_qsize 5, out_qsize 0
2017-11-12 16:14:32,662 : INFO : PROGRESS: at 98.95% examples, 308753 wor

In [16]:
# done with training, free up some memory
model.delete_temporary_training_data(keep_inference=True)

In [17]:
model.save('reviews.d2v')
# in other program, we could write: model = Doc2Vec.load('reviews.d2v')

2017-11-12 16:14:35,953 : INFO : saving Doc2Vec object under reviews.d2v, separately None
2017-11-12 16:14:35,956 : INFO : not storing attribute syn0norm
2017-11-12 16:14:35,958 : INFO : not storing attribute cum_table
2017-11-12 16:14:40,700 : INFO : saved reviews.d2v


In [18]:
model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))

array([ 0.16600764,  0.29806057, -0.37614173,  0.58661956,  0.31548923,
       -0.15109532, -0.19294184, -0.80975324, -0.13256417, -0.26431978,
        0.15649557, -0.36540538, -0.33639464, -0.55479848, -0.02375498,
        0.12179437, -0.06088163, -0.17349492, -0.19584687,  0.09399831,
        0.01947556, -0.17546433, -0.07536539, -0.05634249,  0.2418247 ,
       -0.11649339, -0.18398936, -0.37568066, -0.04755535, -0.23786636,
        0.35202903, -0.25357839,  0.05126057, -0.22089498,  0.09130105,
       -0.46730992,  0.34186646,  0.17174301,  0.51055247,  0.21438542,
       -0.41699263, -0.5968706 , -0.00541743,  0.39446551,  0.07960459,
       -0.20494871,  0.11499975,  0.22761559,  0.24039924, -0.06279976], dtype=float32)

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))],
    [model.infer_vector(extract_words("Service sucks."))])

array([[ 0.48211202]], dtype=float32)

In [20]:
cosine_similarity(
    [model.infer_vector(extract_words("Highly recommended."))],
    [model.infer_vector(extract_words("Service sucks."))])

array([[ 0.28899333]], dtype=float32)

In [21]:
sentences = []
sentvecs = []
sentiments = []
for fname in ["yelp", "amazon_cells", "imdb"]: 
    with open("sentiment labelled sentences/%s_labelled.txt" % fname, encoding='UTF-8') as f:
        for i, line in enumerate(f):
            line_split = line.strip().split('\t')
            sentences.append(line_split[0])
            words = extract_words(line_split[0])
            sentvecs.append(model.infer_vector(words, steps=10)) # create a vector for this document
            sentiments.append(int(line_split[1]))
            
# shuffle sentences, sentvecs, sentiments together
combined = list(zip(sentences, sentvecs, sentiments))
random.shuffle(combined)
sentences, sentvecs, sentiments = zip(*combined)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

clf = KNeighborsClassifier(n_neighbors=9)
clfrf = RandomForestClassifier()

In [23]:
scores = cross_val_score(clf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.75900000000000012, 0.016950909513454807)

In [24]:
scores = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.69766666666666655, 0.019988885800753264)

In [25]:
# bag-of-words comparison
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), RandomForestClassifier())

In [26]:
scores = cross_val_score(pipeline, sentences, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.73733333333333329, 0.015937377450509209)